In [2]:
import argparse

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torchvision import transforms
from ImageDataset import *
from unet import *
from loss import *

## Unzip The Data  
Only if running on google colab


In [8]:
import zipfile
import os

# Specify the path to the uploaded zip file
zip_path = '/content/data.zip'
pred_path = '/predictions.zip'

# Specify the directory where you want to extract the contents

# Create the extraction directory if it doesn't exist

# Extract the contents of the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()
with zipfile.ZipFile(pred_path, 'r') as zip_ref:
    zip_ref.extractall()


FileNotFoundError: ignored

In [9]:
TRAIN_IMAGES = 'data/training/images/'
GROUNDTRUTH = 'data/training/groundtruth/'
TEST_IMAGES = 'data/test_set_images/'
TRAIN_AUG_IMAGES = 'data/augmented/images_1/'
TRAIN_AUG_IMAGES_GT = 'data/augmented/groundtruth_1/'
AUG_IM_DATASET= 'data/augmented/images_for_train_1/'
AUG_GT_DATASET='data/augmented/groundtruth_for_train_1/'

FOREGROUND_TRESHOLD = 0.25
SPLIT_RATIO = 0.9
BATCH_SIZE = 10
EPOCHS = 30
LR = 1e-3
SEED = 0
WEIGHT_DECAY = 1e-3
WORKERS = 2

In [10]:
# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
pin_memory = device == 'cuda'

Device: cuda


In [11]:
image_transform = transforms.Compose([
    transforms.ToTensor(),
])
mask_transform = transforms.Compose([
    transforms.ToTensor(),
])

In [12]:
dataset = ImagesDataset(
    img_dir=TRAIN_IMAGES,
    gt_dir=GROUNDTRUTH,
    image_transform=image_transform,
    mask_transform=mask_transform,
)

In [19]:
dataset_augmented=ImagesDataset(
    img_dir=AUG_IM_DATASET,
    gt_dir=AUG_GT_DATASET,
    image_transform=image_transform,
    mask_transform=mask_transform,
)


In [81]:

len(dataset_augmented)

200

In [82]:
image, mask = dataset[0]
print('Image size:', image.shape)
print('Mask size:', mask.shape)

Image size: torch.Size([3, 400, 400])
Mask size: torch.Size([1, 400, 400])


In [20]:
train_loader_augmented=DataLoader(
    dataset=dataset_augmented,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=WORKERS,
    pin_memory=pin_memory,
)


In [109]:
image, mask = dataset_augmented[55]
print('Image size:', image.shape)
print('Mask size:', mask.shape)

Image size: torch.Size([3, 400, 400])
Mask size: torch.Size([1, 400, 400])


In [21]:
model= UNet().to(device)

In [22]:
criterion = DiceLoss()
optimizer = Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

In [25]:
# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(
    optimizer=optimizer,
    mode='min',
    patience=5,
    verbose=True,
)

In [31]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm


# Set the model in training mode
model.train()

# Define the number of training epochs
epochs = 50  # You can adjust this as needed

# Training loop
for epoch in range(epochs):
    total_loss = 0.0

    # Iterate over the training data
    for data, target in tqdm(train_loader_augmented, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch'):
        # Send the input to the device
        data, target = data.to(device), target.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate the loss
        loss = criterion(output, target)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update the total loss
        total_loss += loss.item()

    # Average loss for the epoch
    average_loss = total_loss / len(train_loader_augmented)
    print(f"average loss: ", average_loss)

    # Adjust learning rate if a scheduler is provided
    if lr_scheduler is not None:
        lr_scheduler.step(average_loss)

torch.save(model.state_dict(), 'trained_model_10ep_30batch.pth')

# Save the trained model
#torch.save(model.state_dict(), 'trained_model.pth')

# Now, you can use the trained model for predictions
# For example, if you have a test DataLoader, you can do:
# model.eval()
# with torch.no_grad():
#     for test_data in test_data_loader:
#         test_data = test_data.to(device)
#         predictions = model(test_data)
#         # Process predictions as needed based on proba_threshold
#         # ...

# Note: This is a basic example, and you might need to adapt it based on your specific requirements and dataset structure.


Epoch 1/50: 100%|██████████| 20/20 [00:10<00:00,  1.91batch/s]


average loss:  0.10535573959350586


Epoch 2/50: 100%|██████████| 20/20 [00:10<00:00,  1.86batch/s]


average loss:  0.10149713158607483


Epoch 3/50: 100%|██████████| 20/20 [00:10<00:00,  1.88batch/s]


...

average loss:  0.0860293060541153


Epoch 49/50: 100%|██████████| 20/20 [00:10<00:00,  1.82batch/s]


average loss:  0.08531491756439209


Epoch 50/50: 100%|██████████| 20/20 [00:10<00:00,  1.83batch/s]

average loss:  0.08559116423130035


## Image Augmentation

In [14]:
# Define custom transformations for satellite images
satellite_image_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=90),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
])

# Define custom transformations for binary masks
binary_mask_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=90),
    transforms.ToTensor(),
])

In [15]:

def create_augmented_dataset():

    # Creates directories
    for dirname in (TRAIN_AUG_IMAGES, TRAIN_AUG_IMAGES_GT):
        os.makedirs(dirname, exist_ok=True)

    # Load the original dataset
    images = sorted(os.listdir(TRAIN_IMAGES))
    masks = sorted(os.listdir(GROUNDTRUTH))

    for i in range(len(images)):
        # Get image and mask names
        image_name = images[i]
        mask_name = masks[i]

        # Get images paths
        image_path = os.path.join(TRAIN_IMAGES, image_name)
        mask_path = os.path.join(GROUNDTRUTH, mask_name)

        # Open images
        image = Image.open(image_path)
        mask = Image.open(mask_path)

        #apply the transformations
        image_transformed = satellite_image_transform(image)
        mask_transformed = binary_mask_transform(mask)

        # Convert tensors to PIL Images
        image_transformed_PIL = transforms.ToPILImage()(image_transformed)
        mask_transformed_PIL = transforms.ToPILImage()(mask_transformed)

        # Save augmented images
        filename_img = f'Image_{i+1:04d}.png'
        filename_gd = f'gdImage_{i+1:04d}.png'
        image_path = os.path.join(TRAIN_AUG_IMAGES, filename_img)
        mask_path = os.path.join(TRAIN_AUG_IMAGES_GT, filename_gd)
        image_transformed_PIL.save(image_path)
        mask_transformed_PIL.save(mask_path)




In [16]:
#create augmented dataset
create_augmented_dataset()

In [56]:
import os
import shutil

def copy_contents(source, destination):
    """
    Copy contents from source to destination.

    Parameters:
        source (str): Path to the source directory.
        destination (str): Path to the destination directory.
    """
    for item in os.listdir(source):
        source_path = os.path.join(source, item)
        destination_path = os.path.join(destination, item)
        shutil.copy(source_path, destination_path)

def create_combined_directory(directory1, directory2, combined_directory):
    """
    Create a new directory with copies of contents from two directories.

    Parameters:
        directory1 (str): Path to the first directory.
        directory2 (str): Path to the second directory.
        combined_directory (str): Path to the new directory where contents will be copied.
    """
    # Create the new directory if it doesn't exist
    os.makedirs(combined_directory, exist_ok=True)

    # Copy contents from directory1 to the combined directory
    copy_contents(directory1, combined_directory)

    # Copy contents from directory2 to the combined directory
    copy_contents(directory2, combined_directory)



In [17]:
def merge_directories(directory_1, directory_2, new_directory):
  # Create the new directory if it doesn't exist
  os.makedirs(new_directory, exist_ok=True)

  #Iterate over the contents of the first directory and move them to the new directory
  for item in os.listdir(directory_1):
    source = os.path.join(directory_1, item)
    destination = os.path.join(new_directory, item)
    shutil.move(source, destination)

  # Iterate over the contents of the second directory and move them to the new directory
  for item in os.listdir(directory_2):
    source = os.path.join(directory_2, item)
    destination = os.path.join(new_directory, item)
    shutil.move(source, destination)

In [18]:
import shutil

merge_directories(TRAIN_AUG_IMAGES,TRAIN_IMAGES, AUG_IM_DATASET)
merge_directories(TRAIN_AUG_IMAGES_GT,GROUNDTRUTH,AUG_GT_DATASET)
#create_combined_directory(TRAIN_AUG_IMAGES,TRAIN_IMAGES, AUG_IM_DATASET)
#create_combined_directory(TRAIN_AUG_IMAGES_GT,GROUNDTRUTH,AUG_GT_DATASET)


## Testing and submission

In [32]:
test_image_transform = transforms.Compose([
    transforms.ToTensor(),
])

In [33]:
test_set = ImagesDataset(
    img_dir=TEST_IMAGES,
    image_transform=test_image_transform,
)

In [34]:
test_loader = DataLoader(
    dataset=test_set,
    num_workers=WORKERS,
    pin_memory=pin_memory,
)

In [35]:
def _get_pred_filename(lenth_loader, index: int) -> str:
    """Returns the filename of the prediction.

    Args:
        index (int): index of the image in the dataset.

    Returns:
        str: filename of the prediction.
    """
    if lenth_loader > 1000:
        return f'prediction_{index + 1:04d}.png'
    return f'prediction_{index + 1:03d}.png'

In [36]:
def _predict_labels(
    output: torch.Tensor,
    proba_threshold: float,
) -> torch.Tensor:
    """Predicts the labels for an output.

    Args:
        output (torch.Tensor): tensor output.
        proba_threshold (float): probability threshold.

    Returns:
        torch.Tensor: tensor of 0 and 1.
    """
    return (output > proba_threshold).type(torch.uint8)

In [37]:
def _save_mask(
    output: torch.Tensor,
    filename: str,
) -> None:
    """Saves the mask as image.

    Args:
        output (torch.Tensor): tensor output.
        filename (str): filename.
        clean (bool, optional): True to clean the prediction using
        postprocessing method. Defaults to True.
    """
    pred_array = torch.squeeze(output * 255).cpu().numpy()
    img = Image.fromarray(pred_array)
    img.save(filename)

In [38]:
predictions_path= 'predictions/'

In [39]:
prediction_filnames = list()

In [ ]:
#Run this only if you load the model
model= UNet().to(device)
state_dict = torch.load('models/trained_model_100ep_10batch.pth', map_location=device)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [44]:
os.makedirs(predictions_path, exist_ok=True)

In [45]:
# Set the model in evaluation mode
model.eval()

# Switch off autograd
with torch.no_grad():
    # Loop over the dataset
    for i, (data, target) in enumerate(test_loader):
        filename = _get_pred_filename(len(test_loader),i)
        print(f'Processing {filename}')

        # Send the input to the device
        data = data.to(device)
        if target.dim() != 1:
            target = target.to(device)

        # Make the predictions
        output = model(data)

        # Get labels
        output = _predict_labels(output, 0.25)

        # Save mask
        output_path = os.path.join(predictions_path, filename)
        _save_mask(output, output_path)
        prediction_filnames.append(output_path)

# Print a message after processing all images
print('Prediction completed.')


Processing prediction_001.png
Processing prediction_002.png
Processing prediction_003.png
Processing prediction_004.png
Processing prediction_005.png
Processing prediction_006.png
Processing prediction_007.png
Processing prediction_008.png
Processing prediction_009.png
Processing prediction_010.png
Processing prediction_011.png
Processing prediction_012.png
Processing prediction_013.png
Processing prediction_014.png
Processing prediction_015.png
Processing prediction_016.png
Processing prediction_017.png
Processing prediction_018.png
Processing prediction_019.png
Processing prediction_020.png
Processing prediction_021.png
Processing prediction_022.png
Processing prediction_023.png
Processing prediction_024.png
Processing prediction_025.png
Processing prediction_026.png
Processing prediction_027.png
Processing prediction_028.png
Processing prediction_029.png
Processing prediction_030.png
Processing prediction_031.png
Processing prediction_032.png
Processing prediction_033.png
Processing

In [46]:
from helper import *
masks_to_submission('submission1.csv', *prediction_filnames)